In [1]:
import numpy as np

from ctypes import c_long

from qiskit_qec.geometry.model.qubit_count import QubitCount
from qiskit_qec.geometry.model.qubit_data import QubitData

from qiskit_qec.geometry.tiles.RSSC import RSSC
from qiskit_qec.geometry.shape import Shape
from qiskit_qec.geometry.plane import Plane
from qiskit_qec.geometry.lattice import Lattice
from qiskit_qec.geometry.tiles.tiling import Tiling
from qiskit_qec.geometry.model.shell import Shell

In [2]:
def make_code(d, manifold):
    # Set parameters for RSSC code on the plane

    # Create the bounding or cutter shape
    cutter = Shape.square(origin=(0,-1), direction=(1,1), length=d-1, manifold=manifold, dtype=int)

    # Set up the qubit counter and aux data structures
    qubit_count = QubitCount()
    qubit_data = QubitData()

    #Choose a tile

    tile = RSSC

    # Basis for Tiling or get from Tile directly
    u = np.array([2,0])
    v = np.array([0,2])

    # Create Tiling lattice
    lattice = Lattice(u,v)

    lattice = lattice.restrict_for_tiling(cutter, tile=tile, expand_value=np.array([0.5,0.5]))

    # Tile the restriced lattice L_l with RSSC tiles
    tiling = Tiling(tile_type=tile, lattice=lattice, qubit_count=qubit_count, qubit_data=qubit_data)

    intersection = cutter.intersection(tiling)
    
    #sh = Shell([tiling.faces[0],tiling.faces[1]])
    
    return intersection
output = make_code(d=3, manifold = Plane())

In [3]:
for face in output.faces.keys():
    if len(output.faces[face][Shape.INSIDE]) == 3:
        print(face.id)

16
65
74
82
123
131
140
189


In [3]:
tiling = Shell()
for face in tiling.faces:
    if face.id in parts[3]:
        tiling.capture_face(face)
    elif face.id in parts[2]:
        del_vertices = 
        face.edit
        tiling.capture_face(face)
        

[[24, 41, 164, 181], [8, 32, 49, 57, 148, 156, 173, 197]]

In [12]:
a = [len(part) for part in intersection.parts]

In [15]:
[len(part) for part in intersection.parts] == [4, 8, 4, 8]

True

In [8]:
a = np.array([1,2])

In [5]:
   
# Cull faces.operators with k<2 qubits inside P

k=2
for i in range(k):
    # Update the auxillary data
    for face in M.faces:
        if face.id in part[i]:
            for vertex in face.vertices:
                qubit_id = qubit_data.qubit[vertex.id]
                qubit_count.decrement_qubit(qubit_id)
      
    # Delete faces and components from shell
    M.delete_subtree(part[i])
    



In [6]:
import numpy as np
from qiskit_qec.geometry.model.vertex import Vertex
from qiskit_qec.geometry.model.edge import Edge
from qiskit_qec.geometry.model.wireframe import WireFrame
from qiskit_qec.geometry.model.face import Face
from qiskit_qec.geometry.model.shell import Shell

In [7]:
 def make_test():
    a = Vertex(np.array([1,1]))
    b = Vertex(np.array([2,2]))
    c = Vertex(np.array([3,3]))

    e1 = Edge([a,b])
    e2 = Edge([b,c])

    wf = WireFrame([e1,e2])
    f = Face(wf)

    s = Shell([f])
    
    
    return s

In [8]:
s = make_test()

In [9]:
def del_test(s):
    tf = s.faces
    s = None
    for face in tf:
        twf = face.wireframe
        face = None
        for wireframe in [twf]:
            tedges = wireframe.edges
            wireframe = None
            for edge in tedges:
                tvertices = edge.vertices
                edge = None
                for vertex in tvertices:
                    vertex = None


In [5]:
levels = 4
partx = [[] for i in range(levels)]

In [7]:
partx[1].append(2)

In [8]:
partx

[[], [2], [], []]